In [4]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os

In [13]:
gmt_url = 'https://raw.githubusercontent.com/kodama222/jarvis/main/data/gmt.csv'
sxp_url = 'https://raw.githubusercontent.com/kodama222/jarvis/main/data/sxp.csv'

urls = [gmt_url, sxp_url]

In [47]:
token_names = ['GMT','SXP']

data_dict = {}

for u in urls:
    
    # take file name
    f0 = os.path.basename(u)
    f1 = f0.rsplit('.')[0].upper()
    
    # read the csv file      
    df = pd.read_csv(u)
    
    # convert date column to datetime and sort by date
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
    df = df.sort_values(by=['date'], ignore_index=True)
    df = df.set_index('date')
    
    df = df.apply(pd.to_numeric)
    
    # create dict entry with dataframe as value and filename as key
    data_dict[f1] = df

In [14]:
sxp = pd.read_csv(sxp_url)
sxp.head()

,date,Private sale,public sale,team,reserve,founder
0,01/07/2019,71119424.25,0.0,1055184.336,0.0,0.0
1,01/08/2019,71119424.25,0.0,2110368.672,0.0,0.0
2,01/09/2019,71119424.25,0.0,3165553.008,0.0,0.0
3,01/10/2019,71119424.25,0.0,4220737.344,0.0,0.0
4,01/11/2019,71119424.25,0.0,5275921.680,0.0,0.0


In [15]:
gmt = pd.read_csv(gmt_url)
gmt.head()

,date,Launchpad sale,private sale,team,advisors,ecosystem/treasury,move and earn
0,01/03/2022,420000000,0,0,0,180000000,0
1,01/04/2022,420000000,0,0,0,198000000,0
2,01/05/2022,420000000,0,0,0,234000000,0
3,01/06/2022,420000000,0,0,0,252000000,0
4,01/07/2022,420000000,0,0,0,270000000,0


In [48]:
data_dict.keys()

dict_keys(['GMT', 'SXP'])

In [20]:
d = list(data_dict.values())

In [24]:
sxp.columns

Index(['date', 'Private sale', 'public sale', 'team', 'reserve', 'founder'], dtype='object')

In [32]:
sxp['Insiders'] = sxp[['Private sale', 'team', 'reserve', 'founder']].sum(axis=1)
sxp['Non-Insiders'] = sxp[['public sale']].sum(axis=1)

In [34]:
sxp.head(10)

,date,Private sale,public sale,team,reserve,founder,Insiders,Non-Insiders
0,01/07/2019,71119424.25,0.00,1.055184e+06,0.0,0.0,7.217461e+07,0.00
1,01/08/2019,71119424.25,0.00,2.110369e+06,0.0,0.0,7.322979e+07,0.00
2,01/09/2019,71119424.25,0.00,3.165553e+06,0.0,0.0,7.428498e+07,0.00
3,01/10/2019,71119424.25,0.00,4.220737e+06,0.0,0.0,7.534016e+07,0.00
4,01/11/2019,71119424.25,0.00,5.275922e+06,0.0,0.0,7.639535e+07,0.00
5,01/12/2019,71119424.25,0.00,6.331106e+06,0.0,0.0,7.745053e+07,0.00
6,01/01/2020,71119424.25,34399009.35,7.386290e+06,0.0,0.0,7.850571e+07,34399009.35
7,01/02/2020,71119424.25,34399009.35,8.441475e+06,0.0,0.0,7.956090e+07,34399009.35
8,01/03/2020,71119424.25,34399009.35,9.496659e+06,0.0,0.0,8.061608e+07,34399009.35
9,01/04/2020,71119424.25,34399009.35,1.055184e+07,0.0,0.0,8.167127e+07,34399009.35


In [36]:
sxp['date'] = pd.to_datetime(sxp['date'], infer_datetime_format=True)
sxp = sxp.sort_values(by=['date'], ignore_index=True)
sxp = sxp.set_index('date')

In [38]:
fig = px.area(sxp.drop(columns=['Insiders', 'Non-Insiders']))
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Token Supply',
    legend_title='Holders',
    plot_bgcolor= 'rgba(0, 0, 0, 0)',
)

fig.show()

In [39]:
fig = px.area(sxp[['Insiders', 'Non-Insiders']])
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Token Supply',
    legend_title='Holders',
    plot_bgcolor= 'rgba(0, 0, 0, 0)',
)

fig.show()

In [40]:
all_final_dist = sxp.drop(columns=['Insiders', 'Non-Insiders']).iloc[-1]
in_vs_nonin_final = sxp[['Insiders', 'Non-Insiders']].iloc[-1]

In [41]:
all_final_dist

Private sale    71119424.25
public sale     34399009.35
team            39041820.43
reserve         56979954.14
founder         52759216.80
Name: 2022-01-07 00:00:00, dtype: float64

In [42]:
in_vs_nonin_final

Insiders        2.199004e+08
Non-Insiders    3.439901e+07
Name: 2022-01-07 00:00:00, dtype: float64

In [43]:
fig = go.Figure(data=[go.Pie(labels=all_final_dist.index, values=all_final_dist.values)])
fig.show()

In [45]:
# Evolution of supply distribution %
fig = px.area(sxp.drop(columns=['Insiders', 'Non-Insiders']),
            title='SXP Supply %',
            groupnorm='fraction')
fig.update_layout(
    yaxis=dict(
            showgrid=True, 
            gridcolor='rgba(166, 166, 166, 0.35)'),
    yaxis_tickformat=',.0%',
    xaxis=dict(
            showgrid=False, 
            gridcolor='rgba(166, 166, 166, 0.35)'),
    xaxis_title='Date',
    yaxis_title='Token Supply',
    
    legend_title='Holders',
    plot_bgcolor= 'rgba(0, 0, 0, 0)',

    autosize=False,
    width=1000,
    height=550,
    title_x=0.4
)

In [49]:
data_dict['SXP']

,private sale,public sale,team,reserve,founder
date,,,,,
2019-01-07,71119424.25,0.00,1.055184e+06,0.000000e+00,0.00
2019-01-08,71119424.25,0.00,2.110369e+06,0.000000e+00,0.00
2019-01-09,71119424.25,0.00,3.165553e+06,0.000000e+00,0.00
2019-01-10,71119424.25,0.00,4.220737e+06,0.000000e+00,0.00
2019-01-11,71119424.25,0.00,5.275922e+06,0.000000e+00,0.00
2019-01-12,71119424.25,0.00,6.331106e+06,0.000000e+00,0.00
2020-01-01,71119424.25,34399009.35,7.386290e+06,0.000000e+00,0.00
2020-01-02,71119424.25,34399009.35,8.441475e+06,0.000000e+00,0.00
2020-01-03,71119424.25,34399009.35,9.496659e+06,0.000000e+00,0.00


In [50]:
sxp_url = 'https://raw.githubusercontent.com/kodama222/jarvis/main/data/sxp.csv'

sxp = pd.read_csv(sxp_url)
sxp.head()

,date,private sale,public sale,team,reserve,founder,insiders,non-insiders
0,01/07/2019,71119424.25,0.0,1055184.336,0.0,0.0,72174608.59,0.0
1,01/08/2019,71119424.25,0.0,2110368.672,0.0,0.0,73229792.92,0.0
2,01/09/2019,71119424.25,0.0,3165553.008,0.0,0.0,74284977.26,0.0
3,01/10/2019,71119424.25,0.0,4220737.344,0.0,0.0,75340161.59,0.0
4,01/11/2019,71119424.25,0.0,5275921.680,0.0,0.0,76395345.93,0.0
